In [1]:
import json, csv
import numpy as np
import pandas as pd

df = pd.read_csv('../datasets/entities_nlp_train_hongyu.csv')
df[20:30:2]

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children,target
20,31,missing,unknown,this is ridiculous....,None,None,None,None,None,None,0
22,33,missing,unknown,Love skiing,None,None,None,None,None,None,0
24,36,missing,unknown,LOOOOOOL,None,None,None,None,None,None,0
26,38,missing,unknown,Was in NYC last week!,"NYC,last week","ORG,DATE",pobj,in,ADP,None,0
28,40,missing,unknown,Cooool :),None,None,None,None,None,None,0


## evaluation

In [2]:
y = df['target']
y.shape

(7613,)

In [17]:
X = df.drop(['target'], axis=1)
X.columns

Index(['id', 'keyword', 'location', 'text', 'entities', 'labels', 'ent_dep',
       'ent_head', 'ent_pos', 'ent_children'],
      dtype='object')

In [15]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
         
    def get_feature_names(self):
        return self.names
    
vec = ColumnTransformer([
#     ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
    ('ent', TfidfVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'entities'),
    ('label', TfidfVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'labels'),
    ('dep', TfidfVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_dep'),
    ('head', TfidfVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_head'),
    ('pos', TfidfVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_pos'),
    ('child', TfidfVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_children'),
    
])

# from sklearn.svm import LinearSVC
# from sklearn.svm import SVC
# clf = SVC(kernel='linear', probability=True)
# clf = SVC(kernel='rbf', probability=True)
# clf = SVC(kernel='poly', probability=True)
# clf = LinearSVC(verbose=True)

from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=20, random_state=0) # use a guassian forest
clf = RandomForestClassifier(n_estimators=200, random_state=0)
# clf = RandomForestClassifier(n_estimators=500, random_state=0) # parallel all jobs


pipeline = make_pipeline(vec, clf)

## evaluation

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=5)
    print('Cross-validation MSE: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


import time
start_time = time.time()

evaluate(pipeline, X, y)

seconds = time.time() - start_time
print('\nEvaluation Time Taken: ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

import time
start_time = time.time()

pipeline.fit(X,y)

seconds = time.time() - start_time
print('Training Time Taken:  ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

              precision    recall  f1-score   support

           0       0.67      0.90      0.77      4342
           1       0.76      0.41      0.53      3271

    accuracy                           0.69      7613
   macro avg       0.72      0.66      0.65      7613
weighted avg       0.71      0.69      0.67      7613

Cross-validation MSE: 0.690 ± 0.066
Training Set Accuracy: 0.995

Evaluation Time Taken:  00:13:53
Training Time Taken:   00:01:41


## random forest

### 500 trees (original features):
```
              precision    recall  f1-score   support

           0       0.67      0.90      0.77      4342
           1       0.76      0.41      0.53      3271

    accuracy                           0.69      7613
   macro avg       0.72      0.66      0.65      7613
weighted avg       0.71      0.69      0.67      7613

Cross-validation MSE: 0.690 ± 0.066
Training Set Accuracy: 0.995

Evaluation Time Taken:  00:13:53
Training Time Taken:   00:01:41
```

### 200 trees (original features):
```
              precision    recall  f1-score   support

           0       0.67      0.91      0.77      4342
           1       0.77      0.41      0.53      3271

    accuracy                           0.69      7613
   macro avg       0.72      0.66      0.65      7613
weighted avg       0.71      0.69      0.67      7613

Cross-validation MSE: 0.692 ± 0.066
Training Set Accuracy: 0.995

Evaluation Time Taken:  00:05:43
Training Time Taken:   00:00:45
```

### 20 trees (original features):
```
              precision    recall  f1-score   support

           0       0.66      0.90      0.76      4342
           1       0.75      0.39      0.51      3271

    accuracy                           0.68      7613
   macro avg       0.71      0.65      0.64      7613
weighted avg       0.70      0.68      0.66      7613

Cross-validation MSE: 0.682 ± 0.060
Training Set Accuracy: 0.987

Evaluation Time Taken:  00:00:49
Training Time Taken:   00:00:05
```

## rbf kernel

```
              precision    recall  f1-score   support

           0       0.65      0.85      0.74      4342
           1       0.67      0.40      0.50      3271

    accuracy                           0.66      7613
   macro avg       0.66      0.63      0.62      7613
weighted avg       0.66      0.66      0.64      7613

Cross-validation MSE: 0.657 ± 0.070
Training Set Accuracy: 0.902

Evaluation Time Taken: 00:09:49
Training Time Taken:   00:01:21
```


## polynomial kernel
```
              precision    recall  f1-score   support

           0       0.64      0.92      0.75      4342
           1       0.74      0.30      0.43      3271

    accuracy                           0.65      7613
   macro avg       0.69      0.61      0.59      7613
weighted avg       0.68      0.65      0.61      7613

Cross-validation MSE: 0.655 ± 0.035
Training Set Accuracy: 0.873

Evaluation Time Taken: 00:11:37
Training Time Taken:   00:01:15
```

# Conclusion:
1. Random forest is the best method so far;
2. Ensemble 500 trees will cause overfitting.

In [16]:
from sklearn.externals import joblib

pipeline.fit(X, y)

rf_mdl = '../models/random-forest-200.pkl'
joblib.dump(pipeline, rf_mdl)
!ls -lSh $rf_mdl

/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


-rw-r--r-- 1 hongyu hongyu 64M Mar 26 00:48 ../models/random-forest-200.pkl
